<a href="https://colab.research.google.com/github/jiwon-hae/CSCI544/blob/main/Official_CSCI544_HW1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Part 1. n-Gram Language Models (30 points)

Create 1-5-gram language models trained on the tiny shakespeare dataset. The relevant probabilities P(word|context) will be stored in Python dictionaries. Do not use any smoothing or back-off (until Question 4). Pay special attention to beginning and end of sequences in the modeling process.

In this section, you may only use libraries imported in the original template.

In [1]:
# Import Libaries
import requests
import collections
import random
import math

In [2]:
url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"
response = requests.get(url)
response.raise_for_status() # Raise an exception for invalid HTTP status codes
text_data = response.text
len(text_data), text_data[:100]

(1115394,
 'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou')

In [3]:
# sample
random.seed(42)

pos = random.randint(0, len(text_data) - 1000)
print(text_data[pos:pos+100])

BY:
Many good morrows to my noble lord!

HASTINGS:
Good morrow, Catesby; you are early stirring
What


In [4]:
# preprocessing - do not change
def preprocess_text(text_data):
  text_data = text_data.replace(',',' , ').replace(';', ' ').replace(':', ' ').replace('.',' . ').replace('?',' ? ').replace('!',' ! ')
  text_data = text_data.replace('-', ' ')
  text_data = text_data.replace('\'', '').replace('"', '')
  text_data = text_data.replace('  ', ' ')
  text_data = text_data.replace('\n\n','\n').replace('\n',' </s> <s> ')
  text_data = '<s> ' + text_data + ' </s>'
  text_data = text_data.lower()
  return text_data

text_data = preprocess_text(response.text)
print(f"Number of words: {len(text_data.split(' '))}")

Number of words: 328097


In [5]:
train_data = text_data[:-10_000]
test_data = text_data[-10_000:]
len(train_data), len(test_data)

(1431030, 10000)

In [6]:
vocab = set(train_data.split(' '))
print(f"Number of unique words: {len(vocab)}")
print(f"Sample unique words: {list(vocab)[:10]}")

Number of unique words: 12124
Sample unique words: ['', 'contributors', 'grains', 'publishd', 'confirm', 'witht', 'govern', 'cooling', 'bechanced', 'hallond']


 ## A. Dealing with Out of Vocabulary Words  (5 Points)

We want a way to handle words that did not appear in the train set. Create a list of out of vocabulary words by identifying words that appear in the train dataset less than 3 times.

**Hint:** You can split the dataset into terms by using corpus.split(' '). You can assume the results will be valid word tokens. No more preprocessing is necessary.

In [7]:
def identify_oov_words(corpus, n=3):
    """
    Identify out-of-vocabulary (OOV) words that appear less than `n` times in the dataset.

    Parameters:
    - dataset: The dataset to process. It should be a dictionary with a 'text' key.
    - n: The frequency threshold below which words are considered OOV.

    Returns:
    - A set of out-of-vocabulary words.
    """
    # INSERT CODE HERE
    counter = {}
    for c in corpus.split():
      if c not in counter:
        counter[c] = 0
      counter[c] += 1

    oov = [key for key, val in counter.items() if val < n]
    return set(oov)

In [8]:
oov_words = identify_oov_words(train_data)

vocab = vocab - oov_words
vocab.add('<UNK>')
print(f"Number of OOV words: {len(oov_words)}")
print(f"Expected number of OOV words: {7181}")

assert len(oov_words) == 7181

Number of OOV words: 7181
Expected number of OOV words: 7181


In [9]:
train_data = ' '.join(['<UNK>' if word not in vocab else word for word in train_data.split(' ')])
test_data = ' '.join(['<UNK>' if word not in vocab else word for word in test_data.split(' ')])

## B. Create the N-Gram Models (12 Points)
In this section, we will train several N-Gram models on ONLY the train_data.

First, calculate the counts of each N-Gram.

In [10]:
uni_counts = collections.defaultdict(lambda:0)
bi_counts = collections.defaultdict(lambda:0)
tri_counts = collections.defaultdict(lambda:0)
four_counts = collections.defaultdict(lambda:0)
five_counts = collections.defaultdict(lambda:0)

In [11]:
# INSERT CODE HERE
token = train_data.split(' ')

def calculate_ngram_counts(dataset, count, n):
  for i in range(len(dataset) - (n - 1)):
    n_gram = tuple(dataset[i:i+n]) if n > 1 else (dataset[i], )
    count[n_gram] += 1

calculate_ngram_counts(token, uni_counts, 1)
calculate_ngram_counts(token, bi_counts, 2)
calculate_ngram_counts(token, tri_counts, 3)
calculate_ngram_counts(token, four_counts, 4)
calculate_ngram_counts(token, five_counts, 5)

Next, implement the N-Gram models themselves.

In [12]:
uni = collections.defaultdict(lambda:0)
bi = collections.defaultdict(lambda:0)
tri = collections.defaultdict(lambda:0)
four = collections.defaultdict(lambda:0)
five = collections.defaultdict(lambda:0)

In [13]:
# INSERT CODE HERE
def build_ngram(n_gram, count, prev_count, n):
  for key in count.keys():
    if n == 1 :
      total_count = sum(count.values())
      n_gram[key[0]] = count[key] / total_count
    else:
      prev = key[:-1]
      n_gram[key] = count[key] / prev_count[prev]

build_ngram(uni, uni_counts, None, 1)
build_ngram(bi, bi_counts, uni_counts, 2)
build_ngram(tri, tri_counts, bi_counts, 3)
build_ngram(four, four_counts, tri_counts, 4)
build_ngram(five, five_counts, four_counts, 5)

In [14]:
# Evaluation
assert five[('<s>', 'against', 'the', 'roman', 'state')] == 1.0 # prob of last given prev 4
assert four[('remain', '</s>', '<s>', 'i')] == 0.25 # prob of last given prev 3
assert tri[('did', 'see', 'and')] == 0.5 # prob of last given prev 2
assert bi[('rash', 'like')] == 0.1 # prob of last given prev 1
assert round(uni[('citizen')],5) == 0.00031 # prob of last

## C. Laplace Smoothing (6 Points)

From now on, we will **focus our attention to the bigram model**. Reimplement the bigram model using Laplacian Smoothing.

In [15]:
def calculate_bigram_probability_with_smoothing(word1, word2):
  # INSERT CODE HERE
  bigram = (word1, word2)
  unigram = (word1, )

  bigram_cnt = bi_counts[bigram] + 1
  unigram_cnt = uni_counts[unigram] + len(vocab)

  return bigram_cnt / unigram_cnt

## D. Evaluate Perplexity (7 Points)

Now let's evaluate the smoothed bigram model quantitively using the intrinsic metric **perplexity**.

Recall perplexity is the inverse probability of the test text
$$\text{ppl}(w_1, \dots, w_n) = p(w_1, \dots, w_n)^{-\frac{1}{N}}$$

For an n-gram model, perplexity is computed by
$$\text{ppl}(w_1, \dots, w_n) = (\prod_i p(w_{i+n}|w_i^{i+n-1})^{-\frac{1}{N}}$$

To get rid of numerical issue, we usually compute through:
$$\text{ppl}(w_1, \dots, w_n) = \exp(-\frac{1}{N}\sum_i \log p(w_{i+n}|w_i^{i+n-1}))$$

Note that you do NOT need to optimize the language model in any way so as to minimize perplexity. Your reported perplexity will have no correlation with your score on this assignment, as far as it is implemented correctly.

**HINT:** Use the calculate_bigram_probability_with_smoothing function created above.

In [16]:
def compute_perplexity(data):
  """
    Computes the perplexity of a given text data using a bigram language model.

    Parameters:
    - data : str
    Returns:
    - float
  """

  assert len(data.split(' ')) >= 5
  # Hint: You should use the math library for exp and log
  # INSERT CODE HERE
  tokens = data.split(' ')
  N = len(tokens)
  log_prob_sum = 0

  for i in range(1, N):
    word1 = tokens[i-1]
    word2 = tokens[i]

    prob = calculate_bigram_probability_with_smoothing(word1, word2)
    log_prob_sum += math.log(prob)

  perplexity = math.exp(-log_prob_sum / N)
  return perplexity

In [17]:
assert round(compute_perplexity(test_data)) == 129

# Part 2. Logistic Regression (40 points)



 In this question, you will be guided to implement logistic regression classifer from scratch. You will use LR classifer to do sentiment analysis task on Twitter dataset (the dataset is provided in the code).

## Import Data

In [18]:
# run this cell to import nltk
import nltk

In [19]:
nltk.download('twitter_samples')
nltk.download('stopwords')
# you only need to run it once

[nltk_data] Downloading package twitter_samples to
[nltk_data]     /Users/jiwon_hae/nltk_data...
[nltk_data]   Package twitter_samples is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jiwon_hae/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [20]:
import numpy as np
import pandas as pd
from nltk.corpus import twitter_samples
import re
import string

from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer

## Prepare the data
* You do not need to split the data. We provide the code for you and you just need to run the code below.


In [21]:
# select the set of positive and negative tweets
all_positive_tweets = twitter_samples.strings('positive_tweets.json')
all_negative_tweets = twitter_samples.strings('negative_tweets.json')

* Train test split: 20% will be in the test set, and 80% in the training set.


In [22]:
# split the data into two pieces, one for training and one for testing (validation set)
test_pos = all_positive_tweets[4000:]
train_pos = all_positive_tweets[:4000]
test_neg = all_negative_tweets[4000:]
train_neg = all_negative_tweets[:4000]

train_x = train_pos + train_neg
test_x = test_pos + test_neg

* Create the numpy array of positive labels and negative labels.

In [23]:
# combine positive and negative labels
train_y = np.append(np.ones((len(train_pos), 1)), np.zeros((len(train_neg), 1)), axis=0)
test_y = np.append(np.ones((len(test_pos), 1)), np.zeros((len(test_neg), 1)), axis=0)

## A.  Text processing (6 points)






*   Remove old style retweet with 'RT' in the sentence
*   Remove hyperlinks

*   Remove hashtag
*   Tokenize the sentence using TweetTokenizer


*   Remove stop words
*   Use PorterStemmer to create stem of words in tweet












In [24]:
def process_tweet(tweet):
  # write your code here
  def remove_rt(tw):
    re_rt = re.compile('RT[\s]+')
    return re_rt.sub(r'', tw)

  def remove_hyperlink(tw):
    re_url = re.compile(r'https?://\S+|www\.\S+')
    return re_url.sub(r'', tw)

  def remove_hashtag(tw):
    re_hashtag = re.compile("#\w+")
    return re_hashtag.sub(r'', tw)

  def remove_stopwords(token):
    s = stopwords.words('english')
    return [i for i in token if i not in s]

  tweet = remove_rt(tweet)
  tweet = remove_hyperlink(tweet)
  tweet = remove_hashtag(tweet)

  tweet_tokenizer = TweetTokenizer(preserve_case = False)
  tokens = tweet_tokenizer.tokenize(tweet)
  filtered_tokens = remove_stopwords(tokens)

  ps = PorterStemmer()
  stemmed_words = [ps.stem(token) for token in filtered_tokens]
  return stemmed_words

We will create a function that will take tweets and their labels as input, go through every tweet, preprocess them, count the occurrence of every word in the data set and create a frequency dictionary.

Notice how the outer for loop goes through each tweet, and the inner for loop steps through each word in a tweet.
The freqs dictionary is the frequency dictionary that's being built.
The key is the tuple (word, label), such as ("happy",1) or ("happy",0). The value stored for each key is the count of how many times the word "happy" was associated with a positive label, or how many times "happy" was associated with a negative label.

In [25]:
def build_freqs(tweets, ys):

    # write your code here
    freqs = {}
    for tw, label in zip(tweets, ys):
      p_tweet = process_tweet(tw)
      label = label.item()

      for t in p_tweet:
        key = (t, label)
        if key not in freqs:
          freqs[key] = 0
        freqs[key] += 1
    return freqs

## B. Logistic regression (16 points)


### Sigmoid (4 points)

You will learn to use logistic regression for text classification.
* The sigmoid function is defined as:

$$ h(z) = \frac{1}{1+\exp^{-z}} \tag{1}$$

It maps the input 'z' to a value that ranges between 0 and 1, and so it can be treated as a probability.


In [26]:
def sigmoid(z):
    '''
    Input:
        z: is the input (can be a scalar or an array)
    Output:
        h: the sigmoid of z
    '''

    # write your code here
    return 1 / (1 + np.exp(-z))

*If you are familiar with Logistic regression, you don't need to go over this instructions. *

### Logistic regression: regression and a sigmoid

Logistic regression takes a regular linear regression, and applies a sigmoid to the output of the linear regression.

Regression:
$$z = \theta_0 x_0 + \theta_1 x_1 + \theta_2 x_2 + ... \theta_N x_N$$
Note that the $\theta$ values are "weights". If you took the Deep Learning Specialization, we referred to the weights with the `w` vector.  In this course, we're using a different variable $\theta$ to refer to the weights.

Logistic regression
$$ h(z) = \frac{1}{1+\exp^{-z}}$$
$$z = \theta_0 x_0 + \theta_1 x_1 + \theta_2 x_2 + ... \theta_N x_N$$
We will refer to 'z' as the 'logits'.

###  Cost function and Gradient

The cost function used for logistic regression is the average of the log loss across all training examples:

$$J(\theta) = -\frac{1}{m} \sum_{i=1}^m y^{(i)}\log (h(z(\theta)^{(i)})) + (1-y^{(i)})\log (1-h(z(\theta)^{(i)}))\tag{5} $$
* $m$ is the number of training examples
* $y^{(i)}$ is the actual label of the i-th training example.
* $h(z(\theta)^{(i)})$ is the model's prediction for the i-th training example.

The loss function for a single training example is
$$ Loss = -1 \times \left( y^{(i)}\log (h(z(\theta)^{(i)})) + (1-y^{(i)})\log (1-h(z(\theta)^{(i)})) \right)$$

* All the $h$ values are between 0 and 1, so the logs will be negative. That is the reason for the factor of -1 applied to the sum of the two loss terms.
* Note that when the model predicts 1 ($h(z(\theta)) = 1$) and the label $y$ is also 1, the loss for that training example is 0.

### Update the weights

To update your weight vector $\theta$, you will apply gradient descent to iteratively improve your model's predictions.  
The gradient of the cost function $J$ with respect to one of the weights $\theta_j$ is:

$$\nabla_{\theta_j}J(\theta) = \frac{1}{m} \sum_{i=1}^m(h^{(i)}-y^{(i)})x_j \tag{5}$$
* 'i' is the index across all 'm' training examples.
* 'j' is the index of the weight $\theta_j$, so $x_j$ is the feature associated with weight $\theta_j$

* To update the weight $\theta_j$, we adjust it by subtracting a fraction of the gradient determined by $\alpha$:
$$\theta_j = \theta_j - \alpha \times \nabla_{\theta_j}J(\theta) $$
* The learning rate $\alpha$ is a value that we choose to control how big a single update will be.


### Implement gradient descent function (12 points)

* The number of iterations `num_iters` is the number of times that you'll use the entire training set.
* For each iteration, you'll calculate the cost function using all training examples (there are `m` training examples), and for all features.
* Instead of updating a single weight $\theta_i$ at a time, we can update all the weights in the column vector:  
$$\mathbf{\theta} = \begin{pmatrix}
\theta_0
\\
\theta_1
\\
\theta_2
\\
\vdots
\\
\theta_n
\end{pmatrix}$$
* $\mathbf{\theta}$ has dimensions (n+1, 1), where 'n' is the number of features, and there is one more element for the bias term $\theta_0$ (note that the corresponding feature value $\mathbf{x_0}$ is 1).
* The 'logits', 'z', are calculated by multiplying the feature matrix 'x' with the weight vector 'theta'.  $z = \mathbf{x}\mathbf{\theta}$
    * $\mathbf{x}$ has dimensions (m, n+1)
    * $\mathbf{\theta}$: has dimensions (n+1, 1)
    * $\mathbf{z}$: has dimensions (m, 1)
* The prediction 'h', is calculated by applying the sigmoid to each element in 'z': $h(z) = sigmoid(z)$, and has dimensions (m,1).
* The cost function $J$ is calculated by taking the dot product of the vectors 'y' and 'log(h)'.  Since both 'y' and 'h' are column vectors (m,1), transpose the vector to the left, so that matrix multiplication of a row vector with column vector performs the dot product.
$$J = \frac{-1}{m} \times \left(\mathbf{y}^T \cdot log(\mathbf{h}) + \mathbf{(1-y)}^T \cdot log(\mathbf{1-h}) \right)$$
* The update of theta is also vectorized.  Because the dimensions of $\mathbf{x}$ are (m, n+1), and both $\mathbf{h}$ and $\mathbf{y}$ are (m, 1), we need to transpose the $\mathbf{x}$ and place it on the left in order to perform matrix multiplication, which then yields the (n+1, 1) answer we need:
$$\mathbf{\theta} = \mathbf{\theta} - \frac{\alpha}{m} \times \left( \mathbf{x}^T \cdot \left( \mathbf{h-y} \right) \right)$$

In [27]:
def gradientDescent(x, y, theta, alpha, num_iters):
    '''
    Input:
        x: matrix of features which is (m,n+1)
        y: corresponding labels of the input matrix x, dimensions (m,1)
        theta: weight vector of dimension (n+1,1)
        alpha: learning rate
        num_iters: number of iterations you want to train your model for
    Output:
        J: the final cost
        theta: your final weight vector
    Hint: you might want to print the cost to make sure that it is going down.
    '''
    ### Write your code here
    m = len(y)
    j_history = []

    for t in range(1, num_iters + 1):
      h = sigmoid(x.dot(theta))

      epsilon = 1e-10
      h = np.clip(h, epsilon, 1 - epsilon)

      j = (1/m) * -np.sum(y * np.log(h) + (1 - y) * np.log(1 - h))
      if np.isnan(j):
        break

      j_history.append(j)

      gradient = (1/m) * x.T.dot(h-y)
      theta -= alpha * gradient

    return j_history[-1], theta

## C. Extracting the features (6 points)

* Given a list of tweets, extract the features and store them in a matrix. You will extract two features.
    * The first feature is the number of positive words in a tweet.
    * The second feature is the number of negative words in a tweet.
* Then train your logistic regression classifier on these features.
* Test the classifier on a validation set.

### Instructions: Implement the extract_features function.
* This function takes in a single tweet.
* Process the tweet using the imported `process_tweet()` function and save the list of tweet words.
* Loop through each word in the list of processed words
    * For each word, check the `freqs` dictionary for the count when that word has a positive '1' label. (Check for the key (word, 1.0)
    * Do the same for the count for when the word is associated with the negative label '0'. (Check for the key (word, 0.0).)

In [28]:
def extract_features(tweet, freqs):
    '''
    Input:
        tweet: a list of words for one tweet
        freqs: a dictionary corresponding to the frequencies of each tuple (word, label)
    Output:
        x: a feature vector of dimension (1,3)
    '''

    # process_tweet tokenizes, stems, and removes stopwords
    word_l = process_tweet(tweet)

    # 3 elements in the form of a 1 x 3 vector
    x = np.zeros((1, 3))

    #bias term is set to 1
    x[0,0] = 1

    # write your code here
    for word in word_l:
      pos_key = (word, 1.0)
      neg_key = (word, 0.0)
      x[0, 1] += freqs[pos_key] if pos_key in freqs else 0
      x[0, 2] += freqs[neg_key] if neg_key in freqs else 0

    return x

## D. Training Your Model (6 points)

To train the model:
* Stack the features for all training examples into a matrix `X`.
* Call `gradientDescent`, which you've implemented above.
* Print the cost J and final weights theta.

In [29]:
# write your code here
m = len(train_x)
X = np.zeros((m, 3))
freqs = build_freqs(train_x, train_y)

for i, tweet in enumerate(train_x):
    X[i, :] = extract_features(tweet, freqs)

y = train_y.reshape(-1, 1)
theta = np.zeros((X.shape[1], 1))

alpha = 0.00001
num_iters = 10000
j, theta = gradientDescent(X, y, theta, alpha, num_iters)
print('cost', j)
print('weights', theta)

cost 0.04665784380621075
weights [[ 0.00067608]
 [ 0.00483756]
 [-0.00590681]]


## E. Test your model (6 points)

It is time for you to test your logistic regression function on some new input that your model has not seen before.

#### Instructions: Write `predict_tweet`
Predict whether a tweet is positive or negative.

* Given a tweet, process it, then extract the features.
* Apply the model's learned weights on the features to get the logits.
* Apply the sigmoid to the logits to get the prediction (a value between 0 and 1).

$$y_{pred} = sigmoid(\mathbf{x} \cdot \theta)$$

In [30]:
def predict_tweet(tweet, freqs, theta):
    '''
    Input:
        tweet: a string
        freqs: a dictionary corresponding to the frequencies of each tuple (word, label)
        theta: (3,1) vector of weights
    Output:
        y_pred: the probability of a tweet being positive or negative
    '''
    # write your code here
    x = extract_features(tweet, freqs)
    z = np.dot(x, theta)
    y_pred = sigmoid(z)
    return y_pred

### Check performance using the test set
After training your model using the training set above, check how your model might perform on real, unseen data, by testing it against the test set.

#### Instructions: Implement `test_logistic_regression`
* Given the test data and the weights of your trained model, calculate the accuracy of your logistic regression model.
* Use your `predict_tweet()` function to make predictions on each tweet in the test set.
* If the prediction is > 0.5, set the model's classification `y_hat` to 1, otherwise set the model's classification `y_hat` to 0.
* A prediction is accurate when `y_hat` equals `test_y`.  Sum up all the instances when they are equal and divide by `m`.

In [31]:
def test_logistic_regression(test_x, test_y, freqs, theta):
    """
    Input:
        test_x: a list of tweets
        test_y: (m, 1) vector with the corresponding labels for the list of tweets
        freqs: a dictionary with the frequency of each pair (or tuple)
        theta: weight vector of dimension (3, 1)
    Output:
        accuracy: (# of tweets classified correctly) / (total # of tweets)
    """

    # write your code here
    m = len(test_x)
    positive_result = 0

    for i in range(m):
      tweet = test_x[i]
      ground_truth = test_y[i]

      y_pred = predict_tweet(tweet, freqs, theta)
      h_hat = 1 if y_pred > 0.5 else 0

      if h_hat == int(ground_truth):
        positive_result += 1

    accuracy = positive_result / m
    return accuracy

In [32]:
tmp_accuracy = test_logistic_regression(test_x, test_y, freqs, theta)
print(f"Logistic regression model's accuracy = {tmp_accuracy:.4f}")

/var/folders/sb/4wckybv97dl1hsflchpbj4080000gn/T/ipykernel_6768/809387247.py:23: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  if h_hat == int(ground_truth):


Logistic regression model's accuracy = 0.9885


#Part 3. Word Embeddings (30 points)

In this question, you will implement cosine similarities from scratch and solve some word analogy problems with pre-trained word vectors.

## A. Load Pre-trained GloVe Word Vectors with Gensim (0 points)

- Gensim is a package that implements the word2vec family of algorithms. Here, we use it to load a pre-trained word vectors named GloVe ([Pennington et al. 2014](https://aclanthology.org/D14-1162/)). Please follow the instructions below.
- See more about the Gensim API here: https://radimrehurek.com/gensim/models/word2vec.html

In [33]:
!pip install gensim

  Using cached scipy-1.10.1-cp311-cp311-macosx_12_0_arm64.whl.metadata (100 kB)
  Using cached numpy-1.24.4-cp311-cp311-macosx_11_0_arm64.whl.metadata (5.6 kB)
  Using cached pandas-1.5.3-cp311-cp311-macosx_11_0_arm64.whl.metadata (11 kB)
Using cached numpy-1.24.4-cp311-cp311-macosx_11_0_arm64.whl (13.8 MB)
Using cached scipy-1.10.1-cp311-cp311-macosx_12_0_arm64.whl (28.7 MB)
Using cached pandas-1.5.3-cp311-cp311-macosx_11_0_arm64.whl (10.8 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
  Attempting uninstall: scipy
    Found existing installation: scipy 1.13.1
    Uninstalling scipy-1.13.1:
      Successfully uninstalled scipy-1.13.1
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --

In [35]:
import gensim.downloader
import numpy as np

ImportError: cannot import name 'prod' from 'scipy._lib._util' (/opt/anaconda3/lib/python3.11/site-packages/scipy/_lib/_util.py)

In [ ]:
glove_vectors = gensim.downloader.load('glove-wiki-gigaword-50')

In [ ]:
print('data type:', type(glove_vectors.vectors))
print('# words:', glove_vectors.vectors.shape[0])
print('Embedding dimension:', glove_vectors.vectors.shape[1])

## B. Given a query word, find the top 10 words in the vocabulary that have the highest cosine similarity scores (10 points)

In [ ]:
glove_vectors.most_similar('cat', topn=10)

## You need to implement `my_most_similar()` that will do the same thing as the built-in `most_similar()` function above
- cosine similarity: $\frac{v_1 \cdot v_2}{\lVert v_1 \rVert \lVert v_2 \rVert}$
- $\lVert v_1 \rVert$ means the vector norm of $v_1$
- We have a vocabulary of 400000 words. You should calculate the cosine similarity between `vec(query_word)` and `vec(any other words)`.
- Find the top 10 words that have the highest cosine similarity with the query word.
- Return a list of tuple, where each tuple contains a word (str) and its corresponding cosine similarity score (float) to the query word.
- The following functions are the only built-in functions you are allowed to use for all the questions in Part 3:
  - `glove_vectors.get_index()`, `glove_vectors.get_vector()`, `glove_vectors.index_to_key` (See how to use these functions below.)
  - `np.dot()`, `np.sum()`, `np.argsort(), np.transpose()`
- Hint: Do **not** include the query word itself in the top 10 words
- Hint: To implement $\sqrt{N}$, you can do `N**0.5`

In [ ]:
glove_vectors.get_index('apple') # get the index of a word in the vocabulary

In [ ]:
glove_vectors.index_to_key[3292] # index to word

In [ ]:
glove_vectors.get_vector(word) # get the word vector of a word

In [ ]:
def my_most_similar(glove_vectors, query_word, topn):
    """
    Find the most similar words to a given query word based on cosine similarity in the GloVe embedding space.

    Args:
        glove_vectors (Gensim KeyedVectors)
        query_word (str): The word for which to find the most similar words.
        topn (int): The number of most similar words to return.

    Returns:
        list of tuples:
            - Each tuple contains a word (str) and its corresponding cosine similarity score (float) to the query word.
            - The list is sorted in descending order of cosine similarity.
    """
    # W (numpy.ndarray): glove word embeddings of shape (400000, 50)
    W = glove_vectors.vectors

    # Your code here
    def normalize(X):
        squared_sums = np.sum(X * X, axis=1) if X.ndim > 1 else np.sum(X * X)
        norms = squared_sums ** 0.5
        return norms
    
    query_vector = glove_vectors.get_vector(query_word)
    query_idx = glove_vectors.get_index(query_word)
    
    dot_product = np.dot(W, query_vector)    
    W_norm = normalize(W)
    query_vector_norm = normalize(query_vector)
    cosine_similarities = dot_product / (W_norm * query_vector_norm)
    
    # query_vector_norm = query_vector / normalize(query_vector)
    # W_norm = W / normalize(W)[:, np.newaxis]
    # cosine_similarities = np.dot(W_norm, query_vector_norm)
    
    W_norm_1 = np.linalg.norm(W, axis=1)
    query_vector_norm_1 = np.linalg.norm(query_vector)
    cosine_similarities_1 = dot_product / (W_norm_1 * query_vector_norm_1)
    
    assert(np.equal(W_norm == W_norm_1))
    assert(np.equal(query_vector_norm, query_vector_norm_1))
    assert(np.equal(cosine_similarities, cosine_similarities_1))
    
    
    
    most_similar_indices = np.argsort(-cosine_similarities)

    my_list = [(glove_vectors.index_to_key[i], cosine_similarities[i])
               for i in most_similar_indices if i != query_idx][:topn]

    assert len(my_list) == topn
    return my_list

## C. What are the highest and lowest vector norms among all the word vectors? Answer with two values. (4 points)

In [ ]:
#Your code here
W = glove_vectors.vectors
def normalize(X):
    squared_sums = np.sum(X * X, axis = 1)
    norms = squared_sums ** 0.5
    return norms

norms = normalize(W)
max_vector_norm = -float('inf')
min_vector_norm = float('inf')
for norm in norms:
    if norm > max_vector_norm:
        max_vector_norm = norm
    if norm < min_vector_norm:
        min_vector_norm = norm
        
print(f'max_vector_norm: {max_vector_norm:.3f}, min_vector_norm: {min_vector_norm:.3f}')

## D. Why is cosine similarity better than dot product when calculating similarities between word vectors? (3 points)

Cosine similarity is better than dot product when calculating similarities between word vectors because it measures the angle between vectors, rather than their magnitudes. This makes it more effective at comparing vectors of different lengths, as it focuses on the direction of the vectors, not their size. In contrast, the dot product is influenced by both the direction and magnitude, which can distort the comparison, especially when dealing with documents of varying word counts. 

## Your implementation of `my_most_similar()` should pass the test cases below

In [ ]:
def diff_results(oracle_list, my_list):
  for oracle, mine in zip(oracle_list, my_list):
    assert oracle[0] == mine[0], "find the wrong word"
    assert np.isclose(oracle[1], mine[1]), "wrong consine similarity"

for query in ['computer', 'frog', 'car']:
  oracle_list = glove_vectors.most_similar(query, topn=10)
  my_list = my_most_similar(glove_vectors, query, topn=10)
  diff_results(oracle_list, my_list)

## E. We visualize the word embeddings with PCA below. What do you see in the figure? (3 points)
- Hint: Each dot corresponds to a word vector. Do you observe any meaningful direction between related words?
- You do not need to write any code in this question

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
Z = pca.fit_transform(glove_vectors.vectors)

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(10,5))
for word in ['king', 'queen', 'lord', 'lady', 'prince', 'princess', 'men', 'women']:
  point = Z[glove_vectors.get_index(word)]
  plt.scatter(point[0], point[1], color='b')
  plt.annotate(word, (point[0], point[1]))
  plt.xticks([])
  plt.yticks([])

I observe that words typically associated with males, such as ‘king’, ‘prince’, and ‘lord’, are clustered in the top-left of the plot, while female-associated words, like ‘lady’, ‘princess’, and ‘queen’, are grouped together in the bottom-left. Also, word pairs that represent gender counterparts, such as ‘king’ and ‘queen’ or ‘prince’ and ‘princess’ or ‘lord’ and ‘lady’, are positioned somewhat opposite to each other along the y-axis, reflecting their opposing yet related meanings.

## F. Word Analogy: prince is to princess as lord is to? (10 points)
  - word1: `prince`, word2: `princess`, word3: `lord`
  - `direction = vec(word2) - vec(word1)`
  - `vec_tgt = vec(word3) + direction`
  - Calcuate the cosine similarities between `vec_tgt` and all the words in the vocabulary, **except for word3**.
  - Return the word that has the highest cosine similarity score.

In [ ]:
def word_analogy(glove_vectors, word1, word2, word3):
    """
    Args:
        glove_vectors (Gensim KeyedVectors)
        word1 (str): The first word in the analogy.
        word2 (str): The second word in the analogy.
        word3 (str): The third word in the analogy for which to find the analogous word.

    Returns:
        pred_word (str): The word that best completes the analogy.
    """

    # Your code here
    def normalize(X):
        squared_sums = np.sum(X * X, axis=1) if X.ndim > 1 else np.sum(X * X)
        norms = squared_sums ** 0.5
        return norms
    
    v1, v2, v3 = glove_vectors.get_vector(word1), glove_vectors.get_vector(word2), glove_vectors.get_vector(word3)
    vec_tgt = v2 - v1 + v3
    vec_tgt_norm = vec_tgt / normalize(vec_tgt)

    W = glove_vectors.vectors
    W_norm = W / normalize(W)[:, np.newaxis]
    cosine_similarities = np.dot(W_norm, vec_tgt_norm)

    indx_word3 = glove_vectors.get_index(word3)
    indx_highest_similarity = np.argsort([cosine_similarities[i] if i != indx_word3 else -np.inf for i in range(len(cosine_similarities))])[-1]

    pred_word = glove_vectors.index_to_key[indx_highest_similarity]

    print(f'{word1} is to {word2} as {word3} is to? {pred_word}')
    assert pred_word != word3
    return pred_word

In [ ]:
pred_word = word_analogy(glove_vectors, 'prince', 'princess', 'lord')
pred_word = word_analogy(glove_vectors, 'aunt', 'uncle', 'queen')
pred_word = word_analogy(glove_vectors, 'london', 'england', 'paris')
pred_word = word_analogy(glove_vectors, 'cat', 'cats', 'car')